# Tugas Pemrograman Kecil 4

*Author: Febi Imanuela*

For this assignment, you are asked to perform re-ranking experiment with bi-encoder and cross-encoder model on the given dataset. In order to do so, you need to prepare and index the document collection, create the models, and evaluate the performance for further analysis.

---


*   Fill in the `✋TODO` section
*   Provide justification for your choices
*   Report your experiment results and analysis


---


Happy exploring!

# Setup

In [68]:
!pip install python-terrier -q
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git
!pip install faiss-gpu==1.7.2

  Cloning https://github.com/terrierteam/pyterrier_t5.git to /tmp/pip-req-build-d3xyeyxk
  Running command git clone --filter=blob:none --quiet https://github.com/terrierteam/pyterrier_t5.git /tmp/pip-req-build-d3xyeyxk
  Resolved https://github.com/terrierteam/pyterrier_t5.git to commit 8caa9a28e6cd102c8e294a6a01a42a6ff8e41c76
  Preparing metadata (setup.py) ... done
  Created wheel for pyterrier-t5: filename=pyterrier_t5-0.0.1-py3-none-any.whl size=10542 sha256=d9761c1208e3e45e508c70fe7314d1d2282d0659a6efbfe37a8dea848a439221
  Stored in directory: /tmp/pip-ephem-wheel-cache-asyyzlcd/wheels/b4/cf/ee/1ba4e22db2a8d3b697f4f409a9c71f14e8b2aebe2fad5b0836
Successfully built pyterrier-t5


In [2]:
!pip install -U sentence-transformers -q
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 12.5 MB/s eta 0:00:00


In [3]:
!apt-get-update
!apt-get install -y default-jdk

/bin/bash: line 1: apt-get-update: command not found
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  default-jdk-headless default-jre default-jre-headless fonts-dejavu-core fonts-dejavu-extra
  libatk-wrapper-java libatk-wrapper-java-jni libfontenc1 libice-dev libsm-dev libxkbfile1
  libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk openjdk-11-jre x11-utils
Suggested packages:
  libice-doc libsm-doc libxt-doc openjdk-11-demo openjdk-11-source visualvm mesa-utils
The following NEW packages will be installed:
  default-jdk default-jdk-headless default-jre default-jre-headless fonts-dejavu-core
  fonts-dejavu-extra libatk-wrapper-java libatk-wrapper-java-jni libfontenc1 libice-dev libsm-dev
  libxkbfile1 libxt-dev libxtst6 libxxf86dga1 openjdk-11-jdk openjdk-11-jre x11-utils
0 upgraded, 18 newly installed, 0 to remove and 49 not upgraded.
Need to get 5,523 kB of archives.
After this operat

In [4]:
import tarfile
import re
import torch
import random
import numpy as np
import pandas as pd
from io import StringIO
from sentence_transformers import SentenceTransformer, CrossEncoder
from sentence_transformers.util import cos_sim
import pyterrier as pt
from pyterrier.measures import *
if not pt.java.started():
    pt.java.init()

terrier-assemblies 5.10 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


Java started and loaded: pyterrier.java, pyterrier.terrier.java [version=5.10 (build: craigm 2024-08-22 17:33), helper_version=0.0.8]


In [5]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Dataset

ADI: https://ir.dcs.gla.ac.uk/resources/test_collections/adi/

In [6]:
!pip install gdown
!gdown 1LRN6haAEvGLqromHhskNqVWTe8hSdTTN

Downloading...
From: https://drive.google.com/uc?id=1LRN6haAEvGLqromHhskNqVWTe8hSdTTN
To: /content/adi.tar.gz
100% 57.3k/57.3k [00:00<00:00, 3.22MB/s]


In [33]:
with tarfile.open("adi.tar.gz") as tar:
    tar.extractall()

## Qrels

In [34]:
with open("ADI.REL",) as file:
    rel_content = file.read()

In [35]:
qrels_df = pd.read_csv(StringIO(rel_content), delim_whitespace=True, header=None)[[0, 1]]
qrels_df.columns = ["qid", "docno"]
qrels_df["label"] = 1

qrels_df['qid'] = qrels_df['qid'].apply(lambda x: f"Q{x}")
qrels_df['docno'] = qrels_df['docno'].apply(lambda x: f"D{x}")

display(qrels_df)

<ipython-input-35-26452127305c>:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  qrels_df = pd.read_csv(StringIO(rel_content), delim_whitespace=True, header=None)[[0, 1]]


,qid,docno,label
0,Q1,D17,1
1,Q1,D46,1
2,Q1,D62,1
3,Q2,D12,1
4,Q2,D71,1
...,...,...,...
165,Q35,D53,1
166,Q35,D56,1
167,Q35,D59,1
168,Q35,D67,1


## Queries

In [36]:
with open("ADI.QRY",) as file:
    qry_content = file.read()

In [37]:
def extract_qid_pos(qry_content, start):
    start_pos = qry_content.find(".I ", start)
    end_pos = qry_content.find("\n.W\n", start_pos)
    if start_pos != -1:
        start_pos += 3
    return start_pos, end_pos

queries = []
print(qry_content)
qid_start, qid_end = extract_qid_pos(qry_content, 0)
while qid_start != -1:
    qid = f"Q{qry_content[qid_start:qid_end].strip()}"
    next_qid_start, next_qid_end = extract_qid_pos(qry_content, qid_start)
    w_pos = qid_end + 4
    if next_qid_start != -1:
        query = qry_content[w_pos:next_qid_start-4].strip()
    else:
        query = qry_content[w_pos:].strip()
    queries.append({"qid": qid, "query_raw": query})
    qid_start, qid_end = next_qid_start, next_qid_end

queries

.I 1
.W
What problems and concerns are there in making up descriptive titles?  
What difficulties are involved in automatically retrieving articles from 
approximate titles?  
What is the usual relevance of the content of articles to their titles?
.I 2
.W
How can actually pertinent data, as opposed to references or entire articles 
themselves, be retrieved automatically in response to information requests?
.I 3
.W
What is information science?  Give definitions where possible.
.I 4
.W
Image recognition and any other methods of automatically                
transforming printed text into computer-ready form.
.I 5
.W
What special training will ordinary researchers and businessmen need for proper
information management and unobstructed use of information retrieval systems?
What problems are they likely to encounter?
.I 6
.W
What possibilities are there for verbal communication between computers and 
humans, that is, communication via the spoken word?
.I 7
.W
Describe presently working and 

[{'qid': 'Q1',
  'query_raw': 'What problems and concerns are there in making up descriptive titles?  \nWhat difficulties are involved in automatically retrieving articles from \napproximate titles?  \nWhat is the usual relevance of the content of articles to their titles?'},
 {'qid': 'Q2',
  'query_raw': 'How can actually pertinent data, as opposed to references or entire articles \nthemselves, be retrieved automatically in response to information requests?'},
 {'qid': 'Q3',
  'query_raw': 'What is information science?  Give definitions where possible.'},
 {'qid': 'Q4',
  'query_raw': 'Image recognition and any other methods of automatically                \ntransforming printed text into computer-ready form.'},
 {'qid': 'Q5',
  'query_raw': 'What special training will ordinary researchers and businessmen need for proper\ninformation management and unobstructed use of information retrieval systems?\nWhat problems are they likely to encounter?'},
 {'qid': 'Q6',
  'query_raw': 'What pos

## Document

In [38]:
with open("ADI.ALL",) as file:
    doc_content = file.read()

### ✋ TODO

In [39]:
# TODO: you may use the title, the text body, or the combination of both;
# just include the justification of your choice in the report.

# Feel free to perform further data cleaning if needed.
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import contractions

nltk.download('stopwords')

class TextPreprocessor:
    def __init__(self, remove_stopwords=True, stem=False):
        self.remove_stopwords = remove_stopwords
        self.stem = stem
        self.stop_words = set(stopwords.words('english'))
        self.stemmer = PorterStemmer()

        self.keep_words = {'what', 'how', 'why', 'which', 'who', 'when', 'where'}
        self.stop_words = self.stop_words - self.keep_words

    def clean_text(self, text):
        text = str(text)

        # Membuat kata menjadi con't jadi can not
        text = contractions.fix(text)

        text = re.sub(r'[^a-zA-Z\s]', ' ', text)

        text = ' '.join(text.split())

        return text.lower()

    def process_text(self, text):
        text = self.clean_text(text)

        tokens = word_tokenize(text, preserve_line=True)

        if self.remove_stopwords:
            tokens = [token for token in tokens if token not in self.stop_words]

        if self.stem:
            tokens = [self.stemmer.stem(token) for token in tokens]

        return ' '.join(tokens)

def combine_title_text(row, title_weight=1.5):
    title = row['title'] if isinstance(row['title'], str) else ''
    text = row['text_raw'] if isinstance(row['text_raw'], str) else ''

    weighted_title = ' '.join([title] * int(title_weight))

    return f"{weighted_title} {text}".strip()

preprocessor = TextPreprocessor()

def preprocess_documents(docs_df):
    docs_df['combined_text'] = docs_df.apply(combine_title_text, axis=1)

    docs_df['processed_text'] = docs_df['combined_text'].apply(preprocessor.process_text)

    return docs_df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [40]:
"""
Document format:
--------------------
.I <current docno>\n
.T\n<title>\n
.A\n<author>\n
.W\n<text>\n
.I <next docno>
"""

def extract_docno_pos(doc_content, start):
    start_pos = doc_content.find(".I ", start)
    end_pos = doc_content.find("\n.T\n", start_pos)
    if start_pos != -1:
        start_pos += 3
    return start_pos, end_pos

def extract_author_start(doc_content, start, end):
    return doc_content.find("\n.A\n", start, end)

def extract_text_start(doc_content, start, end):
    return doc_content.find("\n.W\n", start, end)

docs = []
docno_start, docno_end = extract_docno_pos(doc_content, 0)
while docno_start != -1:
    """ Current and Next Docno """
    docno = f"D{doc_content[docno_start:docno_end].strip()}"
    next_docno_start, next_docno_end = extract_docno_pos(doc_content, docno_start)

    """ Title """
    t_start = docno_end + 4
    t_end = extract_author_start(doc_content, t_start, next_docno_start)
    if t_end == -1:
        a_start = -1
        t_end = extract_text_start(doc_content, t_start, next_docno_start)
    else:
        a_start = t_end + 4
    title = f"{doc_content[t_start:t_end].strip()}"

    """ Author (may or may not exist) """
    if a_start == -1:
        author = ""
        d_start = t_end + 4
    else:
        a_end = extract_text_start(doc_content, a_start, next_docno_start)
        author = f"{doc_content[a_start:a_end].strip()}"
        d_start = a_end + 4

    """ Text """
    if next_docno_start != -1:
        text = doc_content[d_start:next_docno_start-4].strip()
    else:
        text = doc_content[d_start:].strip()

    docs.append({"docno": docno, "author": author, "title": title, "text_raw": text})
    docno_start, docno_end = next_docno_start, next_docno_end

docs

[{'docno': 'D1',
  'author': 'H. S. WHITE',
  'title': 'the ibm dsd technical information center - a total systems approach\ncombining traditional library features\nand mechanized computer processing',
  'text_raw': "the ibm data systems division technical\n information center (tic) provides an operating developmental\nsystem for integrated and compatible mechanized\n processing of technical information received within the organization.\n  the system offers several advantages :\n     1 . it is a sophisticated mechanized system for dissemination\nand retrieval;\n     2 . it is compatible with all library mechanized\n  records produced under a standard processing format\n  within ibm libraries, providing such traditional tools\n  as 3 x 5 catalog cards, circulation records and overdue\nnotices;\n     3 . it is reversible, so that discontinuation of machine\nprocessing would not cause gaps in the library's\n  manual records;\n     4 . it is controlled, producing statistical evaluations\no

## Impute Missing Data

In [41]:
docs[37]['docno'] = 'D38'
docs[37]['author'] = 'C. M. HERMAN'

## Remove Nonalphanumeric

In [42]:
def remove_nonalphanum(text):
    pattern = re.compile('[\W_]+')
    return pattern.sub(' ', text)

In [43]:
docs_df = pd.DataFrame(docs)
docs_df["text"] = docs_df["text_raw"].apply(remove_nonalphanum).apply(lambda x: x.lower())
docs_df = preprocess_documents(docs_df)
docs_df.tail()

,docno,author,title,text_raw,text,combined_text,processed_text
77,D78,J. F. HORTY\n.A\nT. B. WALSH,use of flexowriters to prepare large amounts o...,rapid and accurate preparation of large\n amou...,rapid and accurate preparation of large amount...,use of flexowriters to prepare large amounts o...,use flexowriters prepare large amounts alphabe...
78,D79,R. W. HARROLD,implementing a large scale retrieval system fo...,the implementation of a large scale military\n...,the implementation of a large scale military i...,implementing a large scale retrieval system fo...,implementing large scale retrieval system sate...
79,D80,B. LIPETZ,design of an experiment for evaluation of the ...,objective evaluation of a documentation\n tech...,objective evaluation of a documentation techni...,design of an experiment for evaluation of the ...,design experiment evaluation citation index re...
80,D81,H. G. BOHNERT\nM. KOCHEN,the automated multilevel encyclopedia as a new...,the computer revolution encourages reexaminati...,the computer revolution encourages reexaminati...,the automated multilevel encyclopedia as a new...,automated multilevel encyclopedia new mode sci...
81,D82,S. ABRAHAM,machine recognition of linguistic synonymy and...,a theory of synonymy of expression has been de...,a theory of synonymy of expression has been de...,machine recognition of linguistic synonymy and...,machine recognition linguistic synonymy logica...


In [44]:
queries_df = pd.DataFrame(queries)
queries_df["query"] = queries_df["query_raw"].apply(remove_nonalphanum).apply(lambda x: x.lower()).apply(preprocessor.process_text)
queries_df.tail()

,qid,query_raw,query
30,Q31,Information systems in the physical sciences.,information systems physical sciences
31,Q32,Attempts at computerized and mechanized system...,attempts computerized mechanized systems gener...
32,Q33,Retrieval systems which provide for the automa...,retrieval systems which provide automated tran...
33,Q34,Methods of coding used in computerized index s...,methods coding used computerized index systems
34,Q35,Government supported agencies and projects dea...,government supported agencies projects dealing...


# Index

## ✋ TODO

In [45]:
# TODO: index the collection, you may perform preprocessing if needed;
# just add the justification in the report

# You need to include the raw document in index's meta, so you can pass it to the encoder later
# Indexing and meta example: https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html#iterdictindexer
!rm -rf ./index

import os

index_path = os.path.abspath("./index/pyterrier")
iter_indexer = pt.IterDictIndexer(index_path, meta={"docno": 20, "processed_text": 1056})

index_ref = iter_indexer.index(docs_df.to_dict(orient="records"))

index = pt.IndexFactory.of(index_ref)

# Model

## ✋ TODO

In [46]:
# Feel free to change the cut-off, just make sure to keep it consistent
CUT_OFF = 30

## BM25

In [47]:
# TODO: Instantiate BM25
bm25 = pt.terrier.Retriever(index, wmodel="BM25")

## BM25 >> Bi-encoder

In [48]:
# TODO: Instantiate bi-encoder
# Feel free to choose the model
# A few examples of the pretrained model: https://sbert.net/docs/sentence_transformer/pretrained_models.html#original-models
bimodel = SentenceTransformer('msmarco-distilbert-base-tas-b')
BATCH_SIZE = 128
from sentence_transformers import util

def _biencoder_apply(dataframe):
    # TODO: Encode the raw document and raw query using sentence transformer,
    # then compute their similarity score
    # Example: https://pyterrier.readthedocs.io/en/latest/text.html#examples-of-sentence-transformers
    query_embs = bimodel.encode(dataframe['query'].values)
    doc_embs = bimodel.encode(dataframe['processed_text'].values)
    scores = cos_sim(query_embs, doc_embs)
    return scores[0]

bi_encT = pt.apply.doc_score(_biencoder_apply, batch_size=BATCH_SIZE)

In [49]:
# TODO: Create a pipeline where the top CUT_OFF result from BM25 is re-ranked with bi-encoder
# To get the raw document from index's meta, use `pt.text.get_text()`, for example:
# (<baseline model> % CUT_OFF) >> pt.text.get_text(<index>, ["<meta, e.g. raw text>"]) >> bi_encT
bm25_biencoder = (bm25 % CUT_OFF) >> pt.text.get_text(index, "processed_text") >> bi_encT

## BM25 >> Cross-encoder

In [50]:
# TODO: Instantiate cross-encoder
# Feel free to choose the model
# A few examples of the pretrained model: https://sbert.net/docs/cross_encoder/pretrained_models.html
crossmodel = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', max_length=512)

def _crossencoder_apply(dataframe):
    # TODO: Use cross-encoder to directly predict the similarity score of the raw document and raw query
    # Example: https://pyterrier.readthedocs.io/en/latest/text.html#examples-of-sentence-transformers
    return crossmodel.predict(list(zip(dataframe['query'].values, dataframe['processed_text'].values)))

cross_encT = pt.apply.doc_score(_crossencoder_apply, batch_size=BATCH_SIZE)

In [51]:
# TODO: Create a pipeline where the top CUT_OFF result from BM25 is re-ranked with cross-encoder
bm25_crossencoder = (bm25 % CUT_OFF) >> pt.text.get_text(index, "processed_text") >> cross_encT

## Bi-encoder >> Cross-encoder

In [52]:
# TODO: Create a pipeline where the top CUT_OFF result from bi-encoder is re-ranked with cross-encoder

# BM25 as first-retrieval to generate a DataFrame with docid or docno
biencoder_crossencoder = bm25 >> pt.text.get_text(index, "processed_text") >> bi_encT % CUT_OFF >> pt.text.get_text(index, ["processed_text"]) >> cross_encT

## BM25 >> MonoT5

In [72]:
from pyterrier_t5 import MonoT5ReRanker

mono_t5 = MonoT5ReRanker("castorini/monot5-base-msmarco", text_field = "processed_text")

bm25_monot5 = (bm25 % CUT_OFF) >> pt.text.get_text(index, "processed_text") >> mono_t5

# Experiment

## ✋ TODO

In [73]:
# TODO: Perform experiment for every model
# (BM25, BM25 >> Bi-encoder, BM25 >> Cross-encoder, and Bi-encoder >> Cross-encoder),
# with significance testing, where the baseline is BM25
# Example: https://pyterrier.readthedocs.io/en/latest/experiments.html#significance-testing

eval_metrics=[P@10, "map", nDCG@10]

pt.Experiment(
    [bm25, bm25_biencoder, bm25_crossencoder, biencoder_crossencoder, bm25_monot5],
    queries_df,
    qrels_df,
    eval_metrics=eval_metrics,
    names=["BM25", "BM25 >> Bi-encoder", "BM25 >> Cross-encoder", "Bi-encoder >> Cross-encoder", "BM25 >> MonoT5"],
    baseline=0
)

pt.apply:   0%|          | 0/7 [00:00<?, ?row/s]

pt.apply:   0%|          | 0/7 [00:00<?, ?row/s]

pt.apply:   0%|          | 0/9 [00:00<?, ?row/s]

pt.apply:   0%|          | 0/7 [00:00<?, ?row/s]

monoT5:   0%|          | 0/234 [00:00<?, ?batches/s]

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


,name,map,P@10,nDCG@10,map +,map -,map p-value,P@10 +,P@10 -,P@10 p-value,nDCG@10 +,nDCG@10 -,nDCG@10 p-value
0,BM25,0.364973,0.202857,0.449196,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BM25 >> Bi-encoder,0.205385,0.157143,0.260837,7.0,26.0,0.000401,7.0,14.0,0.030335,5.0,26.0,0.000151
2,BM25 >> Cross-encoder,0.318294,0.214286,0.420224,13.0,18.0,0.171420,10.0,6.0,0.401599,13.0,15.0,0.379009
3,Bi-encoder >> Cross-encoder,0.279152,0.205714,0.373175,14.0,19.0,0.065873,11.0,9.0,0.855797,15.0,16.0,0.121551
4,BM25 >> MonoT5,0.344227,0.208571,0.440983,15.0,16.0,0.495198,9.0,6.0,0.720643,16.0,11.0,0.799767


In [74]:
# Helper function for analysis if needed
def get_serp_metadata(serp):
    return pd.merge(serp, qrels_df[['qid', 'docno', 'label']], how='left', on=['qid', 'docno'])

def get_serp(observed_qid, model, k, get_qrels_label=True):
    model_result = model.transform(queries_df[queries_df['qid'] == observed_qid])
    if get_qrels_label:
        model_result = get_serp_metadata(model_result)
    top_metadata = model_result[model_result['rank'] < k].sort_values(by=['rank'])
    return top_metadata

# Usage example:
# get_serp(<qid>, <model>, CUT_OFF)

print("BM25: \n\n\n", get_serp("Q31", bm25, CUT_OFF))
print("BM25 >> Bi-encoder: \n\n\n", get_serp("Q31", bm25_biencoder, CUT_OFF))
print("BM25 >> Cross-encoder: \n\n\n", get_serp("Q31", bm25_crossencoder, CUT_OFF))
print("Bi-encoder >> Cross-encoder: \n\n\n", get_serp("Q31", biencoder_crossencoder, CUT_OFF))

BM25: 


     qid  docid docno  rank     score  \
0   Q31     59   D60     0  8.544085   
1   Q31     42   D43     1  4.889418   
2   Q31     39   D40     2  3.951928   
3   Q31      9   D10     3  3.823036   
4   Q31      2    D3     4  3.551502   
5   Q31     45   D46     5  3.442008   
6   Q31     20   D21     6  3.342080   
7   Q31     44   D45     7  3.265141   
8   Q31     66   D67     8  2.932443   
9   Q31     71   D72     9  2.790287   
10  Q31     73   D74    10  2.305893   
11  Q31     56   D57    11  2.245910   
12  Q31     26   D27    12  1.303303   
13  Q31     51   D52    13  1.282744   
14  Q31     78   D79    14  1.215627   
15  Q31     69   D70    15  1.137271   
16  Q31      0    D1    16  1.121082   
17  Q31     31   D32    17  1.102575   
18  Q31     36   D37    18  1.102575   
19  Q31     61   D62    19  1.091476   
20  Q31     64   D65    20  1.080597   
21  Q31      6    D7    21  1.049226   
22  Q31     19   D20    22  1.025071   
23  Q31     65   D66    23  1.

pt.apply:   0%|          | 0/1 [00:00<?, ?row/s]

BM25 >> Bi-encoder: 


     qid  docid docno     score                                      query_raw  \
22  Q31     19   D20  0.786253  Information systems in the physical sciences.   
0   Q31     59   D60  0.782744  Information systems in the physical sciences.   
2   Q31     39   D40  0.772622  Information systems in the physical sciences.   
1   Q31     42   D43  0.767238  Information systems in the physical sciences.   
4   Q31      2    D3  0.763202  Information systems in the physical sciences.   
5   Q31     45   D46  0.761247  Information systems in the physical sciences.   
6   Q31     20   D21  0.760165  Information systems in the physical sciences.   
7   Q31     44   D45  0.755094  Information systems in the physical sciences.   
16  Q31      0    D1  0.745257  Information systems in the physical sciences.   
3   Q31      9   D10  0.743944  Information systems in the physical sciences.   
21  Q31      6    D7  0.741055  Information systems in the physical sciences.   
11  

pt.apply:   0%|          | 0/1 [00:00<?, ?row/s]

BM25 >> Cross-encoder: 


     qid  docid docno      score  \
22  Q31     19   D20  -3.753677   
2   Q31     39   D40  -4.353572   
4   Q31      2    D3  -5.170814   
0   Q31     59   D60  -5.430927   
1   Q31     42   D43  -6.629130   
3   Q31      9   D10  -6.952489   
16  Q31      0    D1  -7.060066   
20  Q31     64   D65  -7.318258   
25  Q31     22   D23  -7.338242   
13  Q31     51   D52  -7.816378   
6   Q31     20   D21  -7.848347   
14  Q31     78   D79  -8.619402   
19  Q31     61   D62  -9.006472   
5   Q31     45   D46  -9.348229   
9   Q31     71   D72  -9.825663   
18  Q31     36   D37  -9.834345   
7   Q31     44   D45  -9.985441   
21  Q31      6    D7 -10.501420   
11  Q31     56   D57 -11.024524   
29  Q31     15   D16 -11.055132   
28  Q31     50   D51 -11.066597   
27  Q31     81   D82 -11.157161   
15  Q31     69   D70 -11.181762   
24  Q31     72   D73 -11.182688   
8   Q31     66   D67 -11.237248   
10  Q31     73   D74 -11.311376   
26  Q31     30   D31 -11.327

pt.apply:   0%|          | 0/1 [00:00<?, ?row/s]

pt.apply:   0%|          | 0/1 [00:00<?, ?row/s]

Bi-encoder >> Cross-encoder: 


     qid  docid docno      score  \
18  Q31     19   D20  -3.753677   
2   Q31     39   D40  -4.353572   
4   Q31      2    D3  -5.170814   
0   Q31     59   D60  -5.430927   
1   Q31     42   D43  -6.629130   
3   Q31      9   D10  -6.952489   
14  Q31      0    D1  -7.060066   
16  Q31     64   D65  -7.318258   
21  Q31     22   D23  -7.338242   
24  Q31      5    D6  -7.690567   
11  Q31     51   D52  -7.816378   
6   Q31     20   D21  -7.848347   
12  Q31     78   D79  -8.619402   
15  Q31     61   D62  -9.006472   
5   Q31     45   D46  -9.348229   
9   Q31     71   D72  -9.825663   
7   Q31     44   D45  -9.985441   
17  Q31      6    D7 -10.501420   
27  Q31     80   D81 -10.894774   
25  Q31     76   D77 -10.947603   
10  Q31     56   D57 -11.024524   
23  Q31     49   D50 -11.058060   
22  Q31     50   D51 -11.066597   
28  Q31     16   D17 -11.096669   
13  Q31     69   D70 -11.181762   
20  Q31     72   D73 -11.182688   
29  Q31     10   D11 -